In [1]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--train_dir', type=str, default='./trainDataHJ.txt')
parser.add_argument('--rules_dir', type=str, default='./rulesDataHJ.txt')
parser.add_argument('--support', type=float, default=0.2)
parser.add_argument('--confidence',type=float,default=0.2)
args = parser.parse_args()

In [2]:
y=np.random.randint(1,6,[1000,])
ave_born=np.random.randint(1960,2003,[1000,])
ave_start_work=ave_born+np.random.randint(18,41,[1000,])
ave_start_work[ave_start_work>2020]=2020
dept_num=np.random.randint(10,30,[1000,])
dept_worker_num=dept_num-np.random.randint(2,10,[1000,])
mach_cat=np.random.randint(1,4,[1000,])
meth_cat=np.random.randint(1,4,[1000,])
ave_prof=np.random.random_sample([1000,])

In [4]:
df=pd.DataFrame({
    "ave_born":ave_born,
    "ave_start_work":ave_start_work,
    "dept_num":dept_num,
    "dept_worker_num":dept_worker_num,
    "ave_prof":ave_prof,
    "mach_cat":mach_cat,
    "meth_cat":meth_cat,
    "time":y
})

In [5]:
#df.to_csv('trainDataHJ.csv',index=False)

In [6]:
def data_pre(path):
    df=pd.read_csv(path)
    now_y=datetime.datetime.now().year
    #transform date to time
    df['ave_born']=df['ave_born'].map(lambda x:now_y-x+1)
    df['ave_start_work']=df['ave_start_work'].map(lambda x:now_y-x+1)
    #iscretization
    bins=[18,25,35,45,60]
    df['ave_born']=pd.cut(df['ave_born'],bins,right=False,labels=['1','2','3','4'])
    bins=[1,3,5,10,41]
    df['ave_start_work']=pd.cut(df['ave_start_work'],bins,right=False,labels=['1','2','3','4'])
    bins=[0,0.6,0.8,0.9,1]
    df['ave_prof']=pd.cut(df['ave_prof'],bins,right=False,labels=['1','2','3','4'])
    bins=[0,10,20,31]
    df['dept_num']=pd.cut(df['dept_num'],bins,right=False,labels=['1','2','3'])
    bins=[0,6,16,26,30]
    df['dept_worker_num']=pd.cut(df['dept_worker_num'],bins,right=False,labels=['1','2','3','4'])
    
    #type trans
    df['mach_cat']=df['mach_cat'].apply(str)
    df['meth_cat']=df['meth_cat'].apply(str)
    df['time']=df['time'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [7]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [8]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('time') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [9]:
support=0.05
confidence=0.2
train_dir='trainDataHJ.csv'
rules_dir='rulesDataHJ.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

mach_cat_1&ave_start_work_4 ==> time_2	54	0.2621359223300971

dept_worker_num_3&ave_start_work_4 ==> time_2	59	0.2543103448275862

dept_num_3&dept_worker_num_3&ave_start_work_4 ==> time_2	56	0.25112107623318386

mach_cat_3 ==> time_1	79	0.24382716049382716

ave_born_3 ==> time_3	56	0.24347826086956523

ave_start_work_1 ==> time_1	71	0.24067796610169492

mach_cat_1&ave_prof_1 ==> time_3	50	0.23809523809523808

dept_num_2&dept_worker_num_2&ave_start_work_4 ==> time_5	52	0.2374429223744292

dept_num_3&ave_start_work_4 ==> time_2	66	0.2332155477031802

dept_num_3 ==> time_1	118	0.23274161735700197

meth_cat_3 ==> time_1	71	0.23202614379084968

ave_born_4 ==> time_2	80	0.22988505747126436

dept_num_3&dept_worker_num_3&ave_start_work_4 ==> time_4	51	0.22869955156950672

ave_born_4&ave_start_work_4 ==> time_2	76	0.2261904761904762

dept_num_2&dept_worker_num_2 ==> time_5	87	0.22597402597402597

dept_worker_num_3 ==> time_2	96	0.22588235294117648

mach_cat_1 ==> time_